In [1]:
import pymc as pm
import numpy as np
import arviz as az
import pandas as pd
from pytensor.tensor.subtensor import set_subtensor
import pytensor.tensor as pt

%load_ext lab_black

# Prediction of Time Series*

Adapted from [Unit 10: sunspots.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit10/sunspots.odc).

Data can be found [here](https://raw.githubusercontent.com/areding/6420-pymc/main/data/sunspots.txt).

## Problem statement

Sunspot numbers observed each year from 1770 to 1869. 

BUGS Book Page 258.

In [2]:
y = np.loadtxt("../data/sunspots.txt")

In [3]:
y

array([100.8,  81.6,  66.5,  34.8,  30.6,   7. ,  19.8,  92.5, 154.4,
       125.9,  84.8,  68.1,  38.5,  22.8,  10.2,  24.1,  82.9, 132. ,
       130.9, 118.1,  89.9,  66.6,  60. ,  46.9,  41. ,  21.3,  16. ,
         6.4,   4.1,   6.8,  14.5,  34. ,  45. ,  43.1,  47.5,  42.2,
        28.1,  10.1,   8.1,   2.5,   0. ,   1.4,   5. ,  12.2,  13.9,
        35.4,  45.8,  41.1,  30.4,  23.9,  15.7,   6.6,   4. ,   1.8,
         8.5,  16.6,  36.3,  49.7,  62.5,  67. ,  71. ,  47.8,  27.5,
         8.5,  13.2,  56.9, 121.5, 138.3, 103.2,  85.8,  63.2,  36.8,
        24.2,  10.7,  15. ,  40.1,  61.5,  98.5, 124.3,  95.9,  66.5,
        64.5,  54.2,  39. ,  20.6,   6.7,   4.3,  22.8,  54.8,  93.8,
        95.7,  77.2,  59.1,  44. ,  47. ,  30.5,  16.3,   7.3,  37.3,
        73.9])

In [4]:
t = np.array(range(100))
yr = t + 1770

## Model 1

In [5]:
with pm.Model() as m1:
    eps_0 = pm.Normal("eps_0", 0, tau=0.0001)

    theta = pm.Normal("theta", 0, tau=0.0001)
    c = pm.Normal("c", 0, tau=0.0001)
    sigma = pm.Uniform("sigma", 0, 100)
    tau = 1 / (sigma**2)

    _m = c + theta * pt.roll(y, shift=-1)[:-1]
    m = set_subtensor(_m[0], y[0] - eps_0)

    _eps = y - m
    eps = set_subtensor(_eps[0], eps_0)

    pm.Normal("likelihood", mu=m, tau=tau, observed=y[:-1])

    trace = pm.sample(3000)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [eps_0, theta, c, sigma]


Sampling 4 chains for 1_000 tune and 3_000 draw iterations (4_000 + 12_000 draws total) took 7 seconds.


In [6]:
az.summary(trace)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
eps_0,-0.132,21.244,-39.413,39.804,0.204,0.201,10847.0,7037.0,1.0
theta,0.817,0.060,0.710,0.933,0.001,0.000,8448.0,7955.0,1.0
c,8.539,3.527,1.722,14.830,0.039,0.028,8156.0,8837.0,1.0
sigma,21.941,1.612,18.964,25.010,0.016,0.012,9789.0,7872.0,1.0


## Model 1 using built in AR

Docs are a little lacking here.

further reading (these may be outdated - I haven't had a chance to come back to this yet):

- https://github.com/pymc-devs/pymc4/issues/198
- https://www.pymc-labs.io/blog-posts/bayesian-vector-autoregression/
- https://www.pymc.io/projects/examples/en/latest/time_series/AR.html
- https://discourse.pymc.io/t/vectorized-autoregressive-model/1449

In [7]:
with pm.Model() as m1_ar:
    rho = pm.Normal("rho", 0, tau=0.0001, shape=2)  # shape of rho determines AR order
    sigma = pm.Uniform("sigma", 0, 100)

    # constant=True means rho[0] is the constant term (c from BUGS model)
    pm.AR("likelihood", rho=rho, sigma=sigma, constant=True, observed=y)

    trace = pm.sample(3000, chains=4, cores=1)  # multiprocessing error

/Users/aaron/mambaforge/envs/pymc_env2/lib/python3.11/site-packages/pymc/distributions/timeseries.py:569: UserWarning: Initial distribution not specified, defaulting to `Normal.dist(0, 100, shape=...)`. You can specify an init_dist manually to suppress this warning.
  warnings.warn(
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (4 chains in 1 job)
NUTS: [rho, sigma]


Sampling 4 chains for 1_000 tune and 3_000 draw iterations (4_000 + 12_000 draws total) took 12 seconds.


In [8]:
az.summary(trace)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
rho[0],8.575,3.504,1.806,14.941,0.043,0.030,6745.0,6747.0,1.0
rho[1],0.811,0.059,0.698,0.922,0.001,0.001,6751.0,7155.0,1.0
sigma,21.820,1.563,18.922,24.764,0.018,0.013,7340.0,6499.0,1.0


## Model 2: ARMA(2,1)

Holding off on this for now - supposedly they are working on adding more time series stuff and it looks like a pain to do manually.

In [12]:
%load_ext watermark
%watermark -n -u -v -iv -p pytensor

Last updated: Wed Mar 22 2023

Python implementation: CPython
Python version       : 3.11.0
IPython version      : 8.9.0

pytensor: 2.10.1

arviz   : 0.15.1
pymc    : 5.1.2
pandas  : 1.5.3
numpy   : 1.24.2
pytensor: 2.10.1

